In [15]:
# TORCH.AUTOGRAD를 사용한 자동 미분[원문 보기]
# 신경망을 학습할 때 가장 자주 사용되는 알고리즘은 역전파입니다. 이 알고리즘에서, 매개변수(모델 가중치)는 주어진 매개변수에 대한 손실 함수의 변화도(gradient)에 따라 조정됩니다.

# 이러한 변화도를 계산하기 위해 PyTorch에는 torch.autograd라고 불리는 자동 미분 엔진이 내장되어 있습니다. 이는 모든 계산 그래프에 대한 변화도의 자동 계산을 지원합니다.

# 입력 x, 매개변수 w와 b , 그리고 일부 손실 함수가 있는 가장 간단한 단일 계층 신경망을 가정하겠습니다. PyTorch에서는 다음과 같이 정의할 수 있습니다:


import torch

#input 텐서
x=torch.ones(5) #크기가 5인 1로 채워진 입력 텐서 x를 생성합니다.

#output 텐서
y=torch.zeros(3) #크기가 3인 0으로 채워진 예상 출력 텐서 y를 생성합니다.

#가중치 텐서. 매개변수.
w=torch.randn(5,3,requires_grad=True) #크기가 (5, 3)인 가중치 텐서 w를 생성합니다. 초기값은 랜덤. requires_grad=True는 해당 텐서에 대한 변화도(gradient)를 계산하고 추적(track)해야 함을 나타냅니다.

#편향 텐서. 매개변수. 초기값은 랜덤.
b=torch.randn(3,requires_grad=True)

# 입력 x와 가중치 w의 행렬 곱(matmul)에 편향 b를 더하여 결과 텐서 z를 계산합니다.
z=torch.matmul(x,w)+b

#z와 y 사이의 이진 교차 엔트로피 손실을 계산합니다. 이 함수는 이진 분류 문제에서 로짓(logit) 값과 타깃(target) 값을 입력으로 받아 손실 값을 계산합니다.
loss=torch.nn.functional.binary_cross_entropy_with_logits(z,y)

In [16]:
#이 신경망에서, w와 b는 최적화를 해야 하는 매개변수입니다. 따라서 이러한 변수들에 대한 손실 함수의 변화도를 계산할 수 있어야 합니다. 이를 위해서 해당 텐서에 requires_grad 속성을 설정합니다.

#z의 변화도를 추적하는 연산 그래프의 구성을 출력합니다. 이는 z를 어떤 연산의 결과로서 얻었는지를 보여줍니다. 연산 그래프는 텐서의 연산들을 기반으로 구성되며, 이를 통해 역전파(backpropagation) 과정에서 변화도를 계산
print(f"Gradient function for z = {z.grad_fn}")

#loss의 변화도를 추적하는 연산 그래프의 구성을 출력합니다. 마찬가지로 loss를 어떤 연산의 결과로서 얻었는지를 보여줍니다.
print(f"Gradient function for loss = {loss.grad_fn}")

Gradient function for z = <AddBackward0 object at 0x7f446918e320>
Gradient function for loss = <BinaryCrossEntropyWithLogitsBackward0 object at 0x7f446918f8b0>


In [17]:
# 변화도(Gradient) 계산하기

# 신경망에서 매개변수의 가중치를 최적화하려면 매개변수에 대한 손실함수의 도함수(derivative)를 계산해야 합니다.
# 이러한 도함수를 계산하기 위해, loss.backward() 를 호출한 다음 w.grad와 b.grad에서 값을 가져옵니다:

loss.backward()
# backward() 메서드를 호출하면, 손실 함수에 대한 모델의 파라미터들의 변화도가 계산되어 해당 파라미터들의 .grad 속성에 저장됩니다.
# 이후에 .grad 속성을 통해 파라미터들의 변화도를 확인할 수 있습니다.

# 따라서, loss.backward()를 호출한 후에 w.grad와 b.grad를 출력하면 각각 가중치 w와 편향 b에 대한 변화도를 확인할 수 있습니다.
# 변화도는 손실 함수를 기준으로 파라미터들을 조정해야 할 방향과 크기를 나타내며, 이를 사용하여 경사하강법 등의 최적화 알고리즘을 수행할 수 있습니다.

# loss.backward()를 호출한 후에 변화도(gradient)가 자동으로 계산되고, 이를 사용하여 모델의 매개변수가 업데이트됩니다

print(w.grad)
print(b.grad)

tensor([[0.1552, 0.1812, 0.3326],
        [0.1552, 0.1812, 0.3326],
        [0.1552, 0.1812, 0.3326],
        [0.1552, 0.1812, 0.3326],
        [0.1552, 0.1812, 0.3326]])
tensor([0.1552, 0.1812, 0.3326])


In [18]:
# 변화도 추적 멈추기
#  기본적으로, requires_grad=True인 모든 텐서들은 연산 기록을 추적하고 변화도 계산을 지원합니다.
#  그러나 모델을 학습한 뒤 입력 데이터를 단순히 적용하기만 하는 경우와 같이 순전파 연산만 필요한 경우에는, 이러한 추적이나 지원이 필요 없을 수 있습니다.
#  연산 코드를 torch.no_grad() 블록으로 둘러싸서 연산 추적을 멈출 수 있습니다:

z=torch.matmul(x,w)+b
print(z.requires_grad)

with torch.no_grad():
  z=torch.matmul(x,w)+b
print(z.requires_grad)

True
False


In [20]:
#또다른 방법

z=torch.matmul(x,w)+b
z_det=z.detach()
print(z_det.requires_grad)

False
